In [ ]:
from flask import Flask
from flask_cors import CORS, cross_origin
from draghts import DraughtsBoard, DraughtsPiece
from minimax import best_move
from flask import jsonify
from base import Player
import re
import json

In [ ]:
app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

def board_to_json(board):
    pos = {f"S{square[0]}{square[1]}": piece.name for square, piece in board.position.items() }
    tomove = board.player_to_move.name
    winner = board.winner()
    if winner is not None:
        winner = winner.name
    ret = {'pos': pos, 'tomove': tomove, 'winner': winner}
    json = jsonify(ret)
    return json
    

@app.route('/initial_position')
@cross_origin()
def initial_position():
    b = DraughtsBoard()
    return board_to_json(b)

@app.route('/move/<position_json>')
@cross_origin()
def make_move(position_json):
    position = json.loads(position_json)
    tomove = position['tomove']
    board_pos = position['pos']
    board_pos = {(int(sq[1]), int(sq[2])): DraughtsPiece[piece] for sq, piece in board_pos.items()}
    board = DraughtsBoard(board_pos, Player[tomove])
    move = best_move(board, 2)
    new_board = board.make_move(move)
    
    return(board_to_json(new_board))


In [ ]:
app.run()